# 🛠️ **Building Image classification using pretrained CNN models**
## 1. Overview
This script is designed to fine tune CNNs model on building image dataset.


# 1. Importing Essential Libraries
In this step, we import the necessary tools to build our Deep Learning model.
* **TensorFlow & Keras:** The core framework for building and training neural networks.
* **Pathlib, OS, Json:** For file management and handling paths.
* **CV2, PIL:** For loading and processing images.
* **Matplotlib:** For visualizing data and training graphs.
* **Numpy:** For handling numerical operations and matrices.

In [31]:
#Import all the necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from random import choice
from pathlib import Path
import json
import cv2
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
from random import choice
import os

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import os
os.listdir('/content')

['.config', 'drive', 'sample_data']

# 4. Configuration and Hyperparameters
Here we define the "rules" for our training process:
* **Learning Rate (LR):** Controls how big of a step the model takes when learning.
* **Image Shape:** Resizes all input images to `832x535` pixels.
* **Batch Size:** Processes `10` images at a time (adjust this based on GPU memory).
* **Class Names:** The specific building types we are classifying (`Assam_Type`, `Metal_Sheet`, `RCC`, `Vacant`).
* **File Paths:** Defines where to save the best model weights (`.h5` and `.keras`).

In [34]:
LR = [round(i*0.0001,4) for i in range(1,8)]
print(f'Coarse Learning rates will be chosen from : {LR}')
lr_tune = 1e-5
SEED = 123
Save = True
batch_size = 10
img_shape = (832,535)
epochs = epochs_train = 1
epochs_tune = 2
val_split  = 0.3 # Training vs. Validation split
class_names = ['Assam_Type','Metal_Sheet','RCC','Vacant']

work_foldr = r'/content/drive/MyDrive/AI4DRR Workshop'

Loss='sparse_categorical_crossentropy' # Loss type ##was

data_dir = data_dir_train = f'{work_foldr}/Data' # Directory of images

dir_txt = f'{work_foldr}/Hyper_p_R_E.json' # Latest Hyper Params are stored here

past = f'{work_foldr}'# All previous best are stored here
dir_model1 = f'{work_foldr}/model_E.keras'
dir_model = f'{work_foldr}/model_E.weights.h5'# The model's weights are stored here ## Use in conjunction with .save_weights() and .load_weights()

Coarse Learning rates will be chosen from : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]


# 5. Model Architecture: VGG16 Transfer Learning
We are not building a model from scratch. We use **Transfer Learning**:
1.  **Base:** We load **VGG16**, a powerful pre-trained model that already knows how to recognize shapes and textures.
2.  **Freezing:** We set `layer.trainable = False` to keep VGG16's pre-learned patterns intact.
3.  **Head:** We add our own **Dense layers** (classifiers) on top with `Dropout` (to prevent overfitting) and `L2 Regularization` (to keep weights small and stable).
4.  **Output:** The final layer has `4` neurons (one for each class) with `softmax` activation to output probabilities.

In [35]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def Init_model():
    global img_shape
    global class_names
    classes = len(class_names)

    model = Sequential()

    pretrained_model = tf.keras.applications.Xception(
      include_top=False,
      weights="imagenet",
      input_tensor=None,
      input_shape=None,
      pooling='avg',
      classes=classes,
      classifier_activation="softmax",
      name="xception",
)

    for layer in pretrained_model.layers:
        layer.trainable = False

    model.add(pretrained_model)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax'))

    return model

# 6. Data Pipeline: Splitting and Loading
This function (`img_aug`) handles the critical task of preparing the data:
1.  **Loading:** Uses `image_dataset_from_directory` to load images and automatically assign labels.
2.  **Splitting:** Mathematically divides the data into three sets:
    * **Train:** To teach the model.
    * **Validation:** To tune the model during training.
    * **Test:** To evaluate final performance.
3.  **Batching:** Groups images into batches for faster GPU processing.

In [36]:
def img_aug(data_dir, test_split=0.1):
    global SEED, batch_size, val_split, img_shape, class_names

    # Calculate the actual validation split considering the test split
    actual_val_split = val_split / (1 - test_split)

    # Load the full dataset
    full_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        seed=SEED,
        image_size=img_shape,
        batch_size=None,  # Load without batching first
        class_names=class_names,
    )

    # Print dataset size
    print(f"Total number of samples: {tf.data.experimental.cardinality(full_ds).numpy()}")

    # Split the dataset into train+val and test
    dataset_size = tf.data.experimental.cardinality(full_ds).numpy()
    train_val_size = int((1 - test_split) * dataset_size)
    test_size = dataset_size - train_val_size

    train_val_ds = full_ds.take(train_val_size)
    test_ds = full_ds.skip(train_val_size)

    # Split train+val into train and validation
    train_size = int((1 - actual_val_split) * train_val_size)
    val_size = train_val_size - train_size

    train_ds = train_val_ds.take(train_size)
    val_ds = train_val_ds.skip(train_size)

    # Apply batching
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    test_ds = test_ds.batch(batch_size)

    print(f"Train samples: {train_size}")
    print(f"Validation samples: {val_size}")
    print(f"Test samples: {test_size}")

    return train_ds, val_ds, test_ds

# 7. Helper: Prepare Test Data
A utility function to convert the `Test` dataset from a TensorFlow object into standard NumPy arrays. This makes it easier to inspect images and labels manually later.

In [37]:
def prepare_test_data(test_ds):
    test_images = []
    test_labels = []
    for images, labels in test_ds:
        test_images.append(images.numpy())
        test_labels.append(labels.numpy())

    test_images = np.concatenate(test_images, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)

    return test_images, test_labels

# 8. Storing Hyperparameters
We organize our key settings into a dictionary. This makes it easier to log experiments or save configurations to a JSON file later.

In [38]:
hyper_params = {
    'lr': 0.01 ,#fill as needed if customizing
    'batch_size': batch_size ,
    'img_shape': img_shape, #(832,535)
}

# 9. Load and Split the Data
We call our `img_aug` function defined earlier.
* **Output:** You will see the total count of images and how many were assigned to Training vs. Validation vs. Testing.

In [39]:
# Load and prepare data
train_ds, val_ds, test_ds = img_aug(data_dir)
test_images, test_labels = prepare_test_data(test_ds)

Found 120 files belonging to 4 classes.
Total number of samples: 120
Train samples: 72
Validation samples: 36
Test samples: 12


# 10. Compile the Model
We initialize the model and prepare it for training.
* **Optimizer:** `Adam` (adaptive learning rate).
* **Loss Function:** `sparse_categorical_crossentropy` (used because our labels are integers like 0, 1, 2, 3).
* **Metrics:** We track `accuracy`.

In [40]:
model = Init_model()
train_ds,val_ds,test_ds = img_aug(data_dir_train)

model.compile(
    optimizer=Adam(learning_rate = hyper_params['lr']),
    loss = Loss,
    metrics=['accuracy'],

)
print('Model Setup done successfully')

Found 120 files belonging to 4 classes.
Total number of samples: 120
Train samples: 72
Validation samples: 36
Test samples: 12
Model Setup done successfully


# 11. Initialize History Log
We create a list to store the training performance (loss and accuracy) so we can plot graphs later.

In [41]:
histories = []

# 12. Callbacks: Automation
These tools monitor the training:
* **EarlyStopping:** If the validation loss doesn't improve for 10 epochs, stop training early to save time.
* **ReduceLROnPlateau:** If the model gets stuck, reduce the learning rate to help it find the minimum error.

In [42]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

# 13. Model Checkpoint
This callback saves the **best** version of the model (based on the lowest validation loss). This ensures that even if the model gets worse at the very end of training, we still have the best version saved to disk.

In [43]:
model_chk_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = dir_model,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

# 14. Training and Fine-Tuning
We run the training loop.
* **Strategy:** The loop `range(-1, -2, -1)` unfreezes the **last block** of VGG16.
* **Why?** This allows the model to slightly adjust the high-level features of VGG16 to specifically recognize *your* building types, rather than generic objects.
* **`model.fit`:** Starts the actual learning process.

In [44]:
for j in range(-1,-2,-1):
    model.get_layer("xception").layers[j].trainable = True
    if(j<-1):
        model.get_layer("xception").layers[j+1].trainable = False
    print(f'layer {j} unfrozen')
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs_train,
        callbacks=[early_stopping, lr_scheduler,model_chk_callback]
      )
    histories.append(history)
print('Model Trained Succesfully!(?)')

layer -1 unfrozen
8/8 ━━━━━━━━━━━━━━━━━━━━ 272s 35s/step - accuracy: 0.2185 - loss: 417.4531 - val_accuracy: 0.1944 - val_loss: 16.6747 - learning_rate: 0.0100
Model Trained Succesfully!(?)


# 15. Testing the Model (Inference)
Now that the model is trained, we test it on unseen data:
1.  **Load:** We re-initialize the model structure and load the best saved weights.
2.  **Predict:** We pass the test folder images through the model.
3.  **Decode:** The model outputs probabilities (e.g., `[0.1, 0.8, 0.05, 0.05]`). We use `np.argmax` to find the highest probability and map it back to the text label (e.g., "Metal_Sheet").

In [46]:
# 1. Load model
model = Init_model()
model.load_weights('/content/drive/MyDrive/AI4DRR Workshop/model_E.weights.h5')

# 2. Test folder path
test_dir = '/content/drive/MyDrive/AI4DRR Workshop/Test Data'   # <-- change if needed

# 3. Load test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels=None,
    image_size=img_shape,
    shuffle=False
)

# 4. Predict
predictions = model.predict(test_ds)

# 5. Convert predictions to labels
predicted_indices = np.argmax(predictions, axis=1)
predicted_labels = [class_names[i] for i in predicted_indices]

# 6. Get file paths
file_paths = test_ds.file_paths

# 7. Print results
for file, label in zip(file_paths, predicted_labels):
    print(f"{file} → {label}")


Found 16 files.
1/1 ━━━━━━━━━━━━━━━━━━━━ 43s 43s/step
/content/drive/MyDrive/AI4DRR Workshop/Test Data/112.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/113.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/114.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/25.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/26.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/27.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/35.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/36.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/37.JPG → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/45.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/46.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/47.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/55.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/56.jpg → RCC
/content/drive/MyDrive/AI4DRR Workshop/Test Data/57.jpg → RCC
/content/driv